In [146]:
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np

mnist = tf.keras.datasets.mnist
layer_amt = 3

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

base_model = tf.keras.models.Sequential()
base_model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

for x in range(layer_amt): 
    base_model.add(tf.keras.layers.Dense(128, activation='relu'))

base_model.add(tf.keras.layers.Dropout(0.2))
base_model.add(tf.keras.layers.Dense(10))


base_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
base_model.fit(x_train, y_train, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 2s 991us/step - loss: 0.2440 - accuracy: 0.9273
Epoch 2/15
1875/1875 [==============================] - 2s 914us/step - loss: 0.1057 - accuracy: 0.9674
Epoch 3/15
1875/1875 [==============================] - 2s 921us/step - loss: 0.0774 - accuracy: 0.9762
Epoch 4/15
1875/1875 [==============================] - 2s 863us/step - loss: 0.0603 - accuracy: 0.9813
Epoch 5/15
1875/1875 [==============================] - 2s 854us/step - loss: 0.0511 - accuracy: 0.9840
Epoch 6/15
1875/1875 [==============================] - 2s 893us/step - loss: 0.0425 - accuracy: 0.9863
Epoch 7/15
1875/1875 [==============================] - 2s 954us/step - loss: 0.0370 - accuracy: 0.9884
Epoch 8/15
1875/1875 [==============================] - 2s 946us/step - loss: 0.0313 - accuracy: 0.9897
Epoch 9/15
1875/1875 [==============================] - 2s 889us/step - loss: 0.0284 - accuracy: 0.9911
Epoch 10/15
1875/1875 [==============================] - 2s 862u

In [147]:
base_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.1082 - accuracy: 0.9808 - 156ms/epoch - 498us/step


[0.10823148488998413, 0.9807999730110168]

In [148]:
base_model.save('base_model.keras')

In [149]:
numpy_array = np.random.rand(300,1000)
np.save("array.npy", numpy_array)

In [150]:
# what we need
merge_files('base_model.keras', "array.npy", destination="blahblah.gyatt")

# custom file type "blahblah.optimized"
with open("blahblah.gyatt") as f: 
    base_model, numpy_array = unmerge_files("blahblah.gyatt")

NameError: name 'merge_files' is not defined

In [152]:
import struct
import io
import os

def merge_binary_files(file_a, file_b, file_c):
    with open(file_a, 'rb') as fa, open(file_b, 'rb') as fb:
        content_a = fa.read()
        content_b = fb.read()
        
    # two integer representation 
    metadata = struct.pack('QQ', len(content_a), len(content_b))

    # Write metadata 
    with open(file_c, 'wb') as fc:
        fc.write(metadata)
        fc.write(content_a)
        fc.write(content_b)
    return file_c

def unmerge_binary_file(merged_file):
    if merged_file.split(".")[-1] != "gyatt":
        raise FileNotFoundError("No gyatt file was found.")
    # other directory issues are caught by open method.
    
    output_file_a, output_file_b = "eDJwQf1RYLU5e.keras", "t2XVNhtPV93Z3P2.npy"
    with open(merged_file, 'rb') as f:
        metadata = f.read(struct.calcsize('QQ'))
        size_a, size_b = struct.unpack('QQ', metadata)
        content = f.read()
        content_a = content[:size_a]
        content_b = content[size_a:]
    with open(output_file_a, 'wb') as fa, open(output_file_b, 'wb') as fb:
        fa.write(content_a)
        fb.write(content_b)
    model_ = tf.keras.models.load_model(output_file_a)
    array_ = np.load(output_file_b)
    os.remove(output_file_a)
    os.remove(output_file_b)
    return model_, array_
    
# Example usage
merge_binary_files('base_model.keras', 'array.npy', 'merged_file.gyatt')
comp_model, comp_array = unmerge_binary_file('merged_file.gyatt')

In [153]:
print(comp_array)
print(numpy_array)

[[8.20060499e-01 4.43935612e-02 5.00854690e-01 ... 7.57558948e-01
  3.13012814e-01 3.58139973e-01]
 [2.49778293e-01 6.72454567e-01 5.01881947e-01 ... 6.01865009e-03
  9.33227946e-01 5.02844255e-01]
 [1.54757818e-01 1.57532647e-01 5.00085632e-01 ... 7.34719885e-01
  6.92936300e-01 5.50217773e-01]
 ...
 [4.93852399e-01 7.52928746e-01 3.20060055e-01 ... 8.21144909e-01
  5.49495759e-01 5.20119120e-02]
 [2.53306905e-01 5.51142724e-01 6.30090485e-01 ... 1.57219098e-04
  7.96720529e-01 5.82896234e-01]
 [9.54377511e-01 5.35046457e-01 9.28493503e-01 ... 1.49718752e-01
  4.57464895e-02 4.36774353e-01]]
[[8.20060499e-01 4.43935612e-02 5.00854690e-01 ... 7.57558948e-01
  3.13012814e-01 3.58139973e-01]
 [2.49778293e-01 6.72454567e-01 5.01881947e-01 ... 6.01865009e-03
  9.33227946e-01 5.02844255e-01]
 [1.54757818e-01 1.57532647e-01 5.00085632e-01 ... 7.34719885e-01
  6.92936300e-01 5.50217773e-01]
 ...
 [4.93852399e-01 7.52928746e-01 3.20060055e-01 ... 8.21144909e-01
  5.49495759e-01 5.20119120e-02]

In [154]:
print(comp_array.shape)
np.array_equal(comp_array, numpy_array)

(300, 1000)


True

In [155]:
comp_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.1082 - accuracy: 0.9808 - 170ms/epoch - 542us/step


[0.10823148488998413, 0.9807999730110168]

In [156]:
base_model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.1082 - accuracy: 0.9808 - 121ms/epoch - 388us/step


[0.10823148488998413, 0.9807999730110168]

False

False

False